In [3]:
!pip install -U ginza ja-ginza
!pip install nlplot
!pip install wordcloud

In [8]:
import nlplot
import pandas as pd
import plotly
from plotly.subplots import make_subplots
from plotly.offline import iplot
import matplotlib.pyplot as plt
%matplotlib inline


ImportError: cannot import name 'WordCloud' from 'wordcloud' (C:\twitter\wordcloud.py)

In [4]:
df = pd.read_csv(f'/content/drive/MyDrive/調査資料/百田尚樹/hyakutanaoki.csv',encoding="utf-8")

df["rawContent"]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning:

Columns (21,22) have mixed types.Specify dtype option on import or set low_memory=False.



0                              @mototaka728 わしは勝ったのか負けたのか…
1        ★緊急告知★\r\n\r\n本日、14時30分より、YouTubeライブ配信を行ないます。\...
2        ベンツのSクラスでは自慢にもならんよ。\r\nロールスかベントレーくらいじゃないと。\r\n...
3                                               その車、知りたい！😅
4        ちなみに、大学を中退して43年、同級生に会ったのは、3人だけ。うち2人は偶然。\r\nわしっ...
                               ...                        
52759    @yurinbow先日は有難うございました。百田です。わけのわからない話しかできずに申し訳あ...
52760                    @SAWAYA_fezanいつもお世話になっております。百田です。
52761    @yurindo_ebisu初めまして、百田と申します。検索していたらぶつかりました。影法師...
52762           @yuka24初めまして。検索したらぶつかりました。ご購入予定ありがとうございます＾
52763                         友人に勧められて、アホみたいにtwitteを始めた＾＾；
Name: rawContent, Length: 52764, dtype: object

In [5]:
import spacy
from tqdm import tqdm

nlp = spacy.load("ja_ginza")

words_list = []
for story in tqdm(df["rawContent"]):
  doc = nlp(story)
  words = []
  for token in doc:
    if token.tag_ in ["名詞-普通名詞-一般","動詞-一般","形容詞-一般"]:
      words.append(token.lemma_)
  words_list.append(" ".join(words))

100%|██████████| 52764/52764 [24:55<00:00, 35.29it/s]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df["特徴語"] = pd.NA
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(words_list)
words = vectorizer.get_feature_names()
for book_id, vec in zip(range(len(words_list)), X.toarray()):
  feature_words = []
  for w_id, tfidf in sorted(enumerate(vec), key=lambda x: x[1], reverse=True):
    if tfidf < 0.1:
      break
    feature_word = words[w_id]
    feature_words.append(feature_word)  
  df.at[book_id,"特徴語"] = feature_words
df

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [ ]:
import nlplot
npt = nlplot.NLPlot(df, target_col="特徴語")
stopwords = npt.get_stopword(top_n=0,min_freq=3) + ["こと","いう","もつ","よる","とも","思う","つく","はず","https","co","gt","言う","自分","to","new","the","jnsc","posted","of","for","and","in","you","nikone","san","www"]


In [ ]:
fig_unigram = npt.bar_ngram(
    title='uni-gram',
    xaxis_label='word_count',
    yaxis_label='word',
    ngram=1,
    top_n=50,
    width=800,
    height=1100,
    color=None,
    horizon=True,
    stopwords=stopwords,
    verbose=False,
    save=True,
)
fig_unigram.show()

In [ ]:
# 単語数の分布
fig_histgram = npt.word_distribution(
    title='word distribution',
    xaxis_label='count',
    yaxis_label='',
    width=1000,
    height=500,
    color=None,
    template='plotly',
    bins=None,
    save=True,
)
fig_histgram.show()

In [ ]:
fig_wc = npt.wordcloud(
    width=1000,
    height=600,
    max_words=100,
    max_font_size=100,
    colormap='tab20_r',
    stopwords=stopwords,
    mask_file=None,
    save=True
)
plt.figure(figsize=(15, 25))
plt.imshow(fig_wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# ビルド（データ件数によっては処理に時間を要します）
npt.build_graph(stopwords=stopwords, min_edge_frequency=25)

# ビルド後にノードとエッジの数が表示される。ノードの数が100前後になるようにするとネットワークが綺麗に描画できる


fig_co_network = npt.co_network(
    title='Co-occurrence network',
    sizing=100,
    node_size='adjacency_frequency',
    color_palette='hls',
    width=1100,
    height=700,
    save=True
)
iplot(fig_co_network)

In [ ]:
fig_sunburst = npt.sunburst(
    title='sunburst chart',
    colorscale=True,
    color_continuous_scale='Oryel',
    width=1000,
    height=800,
    save=True
)
fig_sunburst.show()

In [ ]:
fig_treemap = npt.treemap(
    title='Tree map',
    ngram=1,
    top_n=50,
    width=1300,
    height=600,
    stopwords=stopwords,
    verbose=False,
    save=True
)
fig_treemap.show()